In [1]:
import bokeh as bk
import bokeh.models as models
from bokeh.plotting import figure, show
import pandas as pd
import polars as pl

In [2]:
df = pd.DataFrame()
df['x'] = [1,2,3,4,5]
df['y'] = [5,4,3,2,1]

In [3]:
source = models.sources.ColumnDataSource(df)

In [4]:
fig = figure()
fig.scatter(source=source)

GlyphRenderer(id='p1038', ...)

In [29]:
show(fig)

In [7]:
pl.DataFrame(df)

x,y
i64,i64
1,5
2,4
3,3
4,2
5,1
